## 1. PP-OCRv2 Introduction

PP-OCR is a text detection and recognition system released by PaddleOCR for the OCR field. PP-OCRv2 has made some improvements for PP-OCR and constructed a new OCR system. The pipeline of PP-OCRv2 system is as follows (the pink box is the new policy of PP-OCRv2):

<div align="center">
<img src="https://user-images.githubusercontent.com/12406017/200258931-771f5a1d-230c-4168-9130-0b79321558a9.png"  width = "80%"  />
</div>

There are five enhancement strategies:
1. Text detection enhancement strategies: collaborative mutual learning；
2. Text detection enhancement strategies: CcopyPaste data augmentation;
3. Text recognition enhancement strategies: PP-LCNet lightweight backbone network;
4. Text recognition enhancement strategies: UDML knowledge distillation;
5. Text recognition enhancement strategies: enhanced CTC loss;

Compared with PP-OCR, the performance improvement of PP-OCRv2 is as follows:
1. Compared with the PP-OCR mobile version, the accuracy of the model is improved by more than 7%;
2. Compared with the PP-OCR server version, the speed is increased by more than 220%;
3. The total model size is 11.6M, which can be easily deployed on both the server side and the mobile side;

For more details, please refer to the technical report: https://arxiv.org/abs/2109.03144 .

For more information about PaddleOCR, you can click https://github.com/PaddlePaddle/PaddleOCR to learn more.




## 2. Model Effects

The results of PP-OCRv2 are as follows:

<div align="center">
<img src="https://user-images.githubusercontent.com/12406017/200239467-a082eef9-fee0-4587-be48-b276a95bf8d0.gif"  width = "80%"  />
</div>




## 3. How to Use the Model

### 3.1 Inference
* Install PaddleOCR whl package

In [ ]:
! pip install paddleocr --user

* Quick experience

In [ ]:
# command line usage
! wget https://raw.githubusercontent.com/PaddlePaddle/PaddleOCR/dygraph/doc/imgs/11.jpg
! paddleocr --image_dir 11.jpg --use_angle_cls true --ocr_version PP-OCRv2

After the operation is complete, the following results will be output in the terminal:
```log
[[[24.0, 36.0], [304.0, 34.0], [304.0, 72.0], [24.0, 74.0]], ['纯臻营养护发素', 0.964739]]
[[[24.0, 80.0], [172.0, 80.0], [172.0, 104.0], [24.0, 104.0]], ['产品信息/参数', 0.98069626]]
[[[24.0, 109.0], [333.0, 109.0], [333.0, 136.0], [24.0, 136.0]], ['（45元/每公斤，100公斤起订）', 0.9676722]]
......
```




### 3.2 Train the model
The PP-OCR system consists of a text detection model, an angle classifier and a text recognition model. For the three model training tutorials, please refer to the following documents:
1. text detection model: [text detection training tutorial](https://github.com/PaddlePaddle/PaddleOCR/blob/release%2F2.6/doc/doc_ch/detection.md)
1. angle classifier: [angle classifier training tutorial](https://github.com/PaddlePaddle/PaddleOCR/blob/release%2F2.6/doc/doc_ch/angle_class.md)
1. text recognition model: [text recognition training tutorial](https://github.com/PaddlePaddle/PaddleOCR/blob/release%2F2.6/doc/doc_ch/recognition.md)

After the model training is completed, it can be used in series by specifying the model path. The command reference is as follows:
```python
paddleocr --image_dir 11.jpg --use_angle_cls true --ocr_version PP-OCRv2 --det_model_dir=/path/to/det_inference_model --cls_model_dir=/path/to/cls_inference_model --rec_model_dir=/path/to/rec_inference_model
```

## 4. Model Principles

The enhancement strategies are as follows

1. Text detection enhancement strategies
- Adopt CML (Collaborative Mutual Learning) collaborative mutual learning knowledge distillation strategy.
     <div align="center">
   <img src="https://pic4.zhimg.com/80/v2-05f12bcd1784993edabdadbc89b5e9e7_720w.webp"  width = "60%"  />
   </div>

As shown in the figure above, the core idea of CML combines ① the standard distillation of the traditional Teacher guiding Students and ② the direct DML mutual learning of the Students network, which allows the Students network to learn from each other and the Teacher network to guide. Correspondingly, the three key Loss loss functions are carefully designed: GT Loss, DML Loss and Distill Loss. Under the condition that the Teacher network Backbone is ResNet18, it has a good effect on the Student's MobileNetV3.

   - CopyPaste data augmentation
   <div align="center">
   <img src="https://pic1.zhimg.com/80/v2-90239608c554972ac307be07f487f254_720w.webp"  width = "60%"  />
   </div>

Data augmentation is one of the important means to improve the generalization ability of the model. CopyPaste is a novel data augmentation technique, which has been proven effective in object detection and instance segmentation tasks. With CopyPaste, text instances can be synthesized to balance the ratio between positive and negative samples in training images. In contrast, this is impossible with traditional image rotation, random flipping, and random cropping.

The main steps of CopyPaste include: ① randomly select two training images, ② randomly scale jitter and zoom, ③ randomly flip horizontally, ④ randomly select a target subset in one image, and ⑤ paste at a random position in another image. In this way, the sample richness can be greatly improved, and the robustness of the model to the environment can be increased.

2. Text recognition enhancement strategies
- PP-LCNet lightweight backbone network
  
The PP-LCNet with balanced speed and accuracy is adopted to effectively improve the accuracy and reduce the network inference time.

- UDML Knowledge Distillation Strategy
   <div align="center">
   <img src="https://pic1.zhimg.com/80/v2-642d94e092c7d5f90bedbd7c7511636c_720w.webp"  width = "60%"  />
   </div>
On the basis of standard DML knowledge distillation, a supervision mechanism for Feature Map is introduced, Feature Loss is added, the number of iterations is increased, and an additional FC network is added to the Head part, which finally speeds up the distillation and improves the effect.

- Enhanced CTC loss
   <div align="center">
   <img src="https://pic3.zhimg.com/80/v2-864d255454b5196e0a2a916d81ff92c6_720w.webp"  width = "40%"  />
   </div>
   In Chinese text recognition tasks, the problem of misrecognition of the number of similar characters is often encountered. Here, we draw on Metric Learning and introduce Center Loss to further increase the distance between classes to enhance the model's ability to distinguish similar characters. The core idea is shown in the formula above.

## 5. Attention

General data are used in the training process of PP-OCR series models. If the performance is not satisfactory in the actual scene, a small amount of data can be marked for finetune.

## 6. Related papers and citations
```
@article{du2021pp,
  title={PP-OCRv2: bag of tricks for ultra lightweight OCR system},
  author={Du, Yuning and Li, Chenxia and Guo, Ruoyu and Cui, Cheng and Liu, Weiwei and Zhou, Jun and Lu, Bin and Yang, Yehua and Liu, Qiwen and Hu, Xiaoguang and others},
  journal={arXiv preprint arXiv:2109.03144},
  year={2021}
}
```
